# Graph Summarizer

Install the Jaro package through the following command:  
`!pip install jaro-winkler`  
You can reference the installation documentation [here](https://pypi.org/project/jaro-winkler/)

In [1]:
import networkx as nx
import numpy as np
import nltk
import jaro

try:
    from nltk.corpus import stopwords
except:
    import nltk
    nltk.download('stopwords')
finally:
    from nltk.corpus import stopwords

In [25]:
# constants
sw = list(set(stopwords.words('english')))
punct = [
    '!','#','$','%','&','(',')','*',
    '+',',','-','/',':',';','<','=','>','@',
    '[','\\',']','^','_','`','{','|','}','~'
]

## Load Text

I've set up this tutorial in a manner that any body of text can be input to the pipeline. Continue on with this tutorial with the text I've selected or use your own. Below are a set of books you can easily download through the NLTK library and Gutenberg project. All of these books are free to use as they are a part of the public domain.

In [3]:
# free to use books in the public domain which can be fetched by NTLK and project gutenberg
book_choices = [
    'austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt',
    'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt',
    'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt',
    'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt',
    'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt',
    'shakespeare-macbeth.txt', 'whitman-leaves.txt'
]

For the purposes of this tutorial, I will be using Shakespeare's play on Julius Caesar. Do note that this and all other bodies of text written by William Shakespeare is in the [public domain](https://en.wikipedia.org/wiki/Public_domain), as the bodies of text were created prior to the existence of any copyright restrictions [2]. 

In [4]:
try:
    book = nltk.corpus.gutenberg.raw('shakespeare-caesar.txt')
except:
    nltk.download('gutenberg')
finally:
    book = nltk.corpus.gutenberg.raw('shakespeare-caesar.txt')

In [28]:
book[0:2000]

"[The Tragedie of Julius Caesar by William Shakespeare 1599]\n\n\nActus Primus. Scoena Prima.\n\nEnter Flauius, Murellus, and certaine Commoners ouer the Stage.\n\n  Flauius. Hence: home you idle Creatures, get you home:\nIs this a Holiday? What, know you not\n(Being Mechanicall) you ought not walke\nVpon a labouring day, without the signe\nOf your Profession? Speake, what Trade art thou?\n  Car. Why Sir, a Carpenter\n\n   Mur. Where is thy Leather Apron, and thy Rule?\nWhat dost thou with thy best Apparrell on?\nYou sir, what Trade are you?\n  Cobl. Truely Sir, in respect of a fine Workman, I am\nbut as you would say, a Cobler\n\n   Mur. But what Trade art thou? Answer me directly\n\n   Cob. A Trade Sir, that I hope I may vse, with a safe\nConscience, which is indeed Sir, a Mender of bad soules\n\n   Fla. What Trade thou knaue? Thou naughty knaue,\nwhat Trade?\n  Cobl. Nay I beseech you Sir, be not out with me: yet\nif you be out Sir, I can mend you\n\n   Mur. What mean'st thou by tha

## Clean Text

In [6]:
def clean_text(text, sw = sw, punct = punct):
    '''
    This function will clean the input text by lowering, removing certain punctuations, stopwords and 
    new line tags.
    
    params:
        text (String) : The body of text you want to clean
        sw (List) : The list of stopwords you wish to removed from the input text
        punct (List) : The slist of punctuations you wish to remove from the input text
        
    returns:
        This function will return the input text after it's cleaned (the output will be a string)
    '''
    text = text.lower()
    article = text.split(' ')
    # clean stopwords
    article = [x.lstrip().rstrip() for x in article if x not in sw]
    article = [x for x in article if x]
    article = ' '.join(article)
    
    # clean punctuations
    for pun in punct:
        article = article.replace(pun, '')
    
    article = article.replace("[^a-zA-Z]", " ").replace('\r\n', ' ').replace('\n', ' ')
    return article

In [7]:
%time cleaned_book = clean_text(book)

CPU times: user 63.4 ms, sys: 2.07 ms, total: 65.5 ms
Wall time: 64.4 ms


In [8]:
# get sentences
sentences = [x for x in cleaned_book.split('. ') if x not in ['', ' ', '..', '.', '...']]

In [9]:
len(sentences)

1289

## Create Network

In this section we will create an adjacency matrix through the similarity of various sentences. This adjacency matrix will then allow us to create a weighted network (through NetworkX). Please keep in mind the size of your input body of text, since the similarity matrix needs to assign a score for each pair of sentences it will be an extensive and exhausting process for your computer if your body of text is very large (> 2000 sentences).

In the created network, the nodes will be indices associated to the sentences in the book and the edges connecting the sentences will be weighted based on the similarity of a pair of sentences. We can then run the page rank algorithm on that weighted network to identify nodes with a large rank associated to them. 

At very high level, the intuition behind the page rank algorithm is to identify highly popular nodes through the use of [random walks](https://towardsdatascience.com/random-walks-with-restart-explained-77c3fe216bca). It's a very popular network based algorithm Larry Page and was famously used by Google to rank their web pages in their search enginer results [1]. In the context of our example, page rank will tell us highly influential sentences in the network. We can then take the top N most influential sentences, join them together and return them back to the user in the form of a summary of the original body of text. 

In [10]:
def create_similarity_matrix(sentences):
    '''
    The purpose of this function will be to create an N x N similarity matrix.
    N represents the number of sentences and the similarity of a pair of sentences
    will be determined through the Jaro-Winkler Score.
    
    params:
        sentences (List -> String) : This is a list of strings you want to create
                                     the similarity matrix with.
     
    returns:
        This function will return a square numpy matrix
    '''
    
    # identify sentence similarity matrix with Jaro Winkler score
    sentence_length = len(sentences)
    sim_mat = np.zeros((sentence_length, sentence_length))

    for i in range(sentence_length):
        for j in range(sentence_length):
            if i != j:
                similarity = jaro.jaro_winkler_metric(sentences[i], sentences[j])
                sim_mat[i][j] = similarity
    return sim_mat

In [11]:
%time sim_mat = create_similarity_matrix(sentences)

CPU times: user 2min 16s, sys: 335 ms, total: 2min 16s
Wall time: 2min 16s


In [12]:
%%time 
# create network
G = nx.from_numpy_matrix(sim_mat)

CPU times: user 2.35 s, sys: 190 ms, total: 2.54 s
Wall time: 2.69 s


In [13]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1289
Number of edges: 820222
Average degree: 1272.6486


In [14]:
%%time 
# calculate page rank scores
pr_sentence_similarity = nx.pagerank(G)

CPU times: user 10.1 s, sys: 955 ms, total: 11.1 s
Wall time: 11.8 s


The built in page rank function will output a dictionary where the keys are the nodes and the values are the associated page rank score to that node.

In [16]:
ranked_sentences = [
    (sentences[sent], rank) for sent,rank in sorted(pr_sentence_similarity.items(), key=lambda item: item[1], reverse = True)
]

In [18]:
ranked_sentences[0]

('decius well vrgd thinke meet marke antony well beloud caesar should outliue caesar shall finde him a shrewd contriuer',
 0.0008514421863851916)

## Generate Summary

In [22]:
def generate_summary(ranked_sentences, N):
    '''
    This function will generate the summary given a list of ranked sentences and the
    number of sentences the user wants in their summary.
    
    params:
        ranked_sentences (List -> Tuples) : The list of ranked sentences where each
                                            element is a tuple, the first value in the
                                            tuple is the sentence, the second value is
                                            the rank
        N (Integer) : The number of sentences the user wants in the summary
        
    returns:
        This function will return a string associated to the summarized ranked_sentences
        of a book
    '''
    summary = '. '.join([sent[0] for sent in ranked_sentences[0:N]])
    return summary

In [36]:
%%time
N = 25
summary = generate_summary(ranked_sentences, N)

CPU times: user 22 µs, sys: 69 µs, total: 91 µs
Wall time: 94.9 µs


In [37]:
summary

'decius well vrgd thinke meet marke antony well beloud caesar should outliue caesar shall finde him a shrewd contriuer. taper burneth closet sir searching window flint found this paper thus seald vp sure it lye went bed. these as sure do beare fire enough to kindle cowards steele valour the melting spirits women. but compact meane haue vs will prickd number friends or shall on depend you ant. man doth bestride narrow world like colossus petty men walke vnder huge legges peepe about to finde selues dishonourable graues. deepe night crept vpon talke and nature must obey necessitie which niggard little rest there say cassi. noble brutus hath told caesar ambitious if so greeuous fault and greeuously hath caesar answerd it. forget speed antonio to touch calphurnia elders say the barren touched holy chace shake sterrile curse ant. i itching palme you know brutus speakes this or gods speech else last bru. o cassius yoaked lambe that carries anger flint beares fire who much inforced shewes has

## Resources
- [1] https://en.wikipedia.org/wiki/PageRank
- [2] https://en.wikipedia.org/wiki/Public_domain

---